In [1]:
from helper import load_mistral_api_key
api_key, dlai_endpoint = load_mistral_api_key(ret_key=True)

Parse the article with BeautifulSoup

In [2]:
import requests
from bs4 import BeautifulSoup
import re

response = requests.get(
    "https://www.deeplearning.ai/the-batch/issue-245/"
)
html_doc = response.text
soup = BeautifulSoup(html_doc, "html.parser")
tag = soup.find("div", re.compile("^prose--styled"))
text = tag.text
print(text)

Dear friends,Multi-agent collaboration is the last of the four key AI agentic design patterns that I’ve described in recent letters. Given a complex task like writing software, a multi-agent approach would break down the task into subtasks to be executed by different roles — such as a software engineer, product manager, designer, QA (quality assurance) engineer, and so on — and have different agents accomplish different subtasks.Different agents might be built by prompting one LLM (or, if you prefer, multiple LLMs) to carry out different tasks. For example, to build a software engineer agent, we might prompt the LLM: “You are an expert in writing clear, efficient code. Write code to perform the task . . ..” It might seem counterintuitive that, although we are making multiple calls to the same LLM, we apply the programming abstraction of using multiple agents. I’d like to offer a few reasons:It works! Many teams are getting good results with this method, and there’s nothing like results

save the text into a text file

In [3]:
file_name = "AI_greenhouse_gas.txt"
with open(file_name, 'w') as file:
    file.write(text)

Chunking

In [4]:
chunk_size = 512
chunks = [text[i : i + chunk_size] for i in range(0, len(text), chunk_size)]

In [5]:
len(chunks)

37

Get embeddings of the chunks

In [12]:
import os
from mistralai.client import MistralClient


def get_text_embedding(txt):
    client = MistralClient(api_key=api_key, endpoint=dlai_endpoint)
    embeddings_batch_response = client.embeddings(model="mistral-embed", input=txt)
    return embeddings_batch_response.data[0].embedding

print(api_key, dlai_endpoint)

1q2yoYPHBCkAVTu3p3yKwwBoSA0xbqTV https://api.mistral.ai/v1/chat/completions


In [13]:
from helper import get_text_embedding

# Llamar a la función get_text_embedding para obtener los embeddings del texto
embeddings = get_text_embedding(text)

print(api_key, dlai_endpoint)

MistralAPIException: Status: 404. Message: {"detail":"Not Found"}

In [14]:
import numpy as np

text_embeddings = np.array([get_text_embedding(chunk) for chunk in chunks])

MistralAPIException: Status: 404. Message: {"detail":"Not Found"}

In [ ]:
text_embeddings

In [ ]:
len(text_embeddings[0])

Store in a vector database

In [10]:
import faiss

d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

NameError: name 'text_embeddings' is not defined

Embed the user query

In [ ]:
question = "What are the ways that AI can reduce emissions in Agriculture?"
question_embeddings = np.array([get_text_embedding(question)])

In [ ]:
question_embeddings

Search for chunks that are similar to the query

In [8]:
#retrieve
D, I = index.search(question_embeddings, k=2)
print(I)

NameError: name 'index' is not defined

In [ ]:
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
print(retrieved_chunk)

In [ ]:
prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

In [ ]:
from mistralai.models.chat_completion import ChatMessage


def mistral(user_message, model="mistral-small-latest", is_json=False):
    client = MistralClient(api_key=api_key, endpoint=dlai_endpoint)
    messages = [ChatMessage(role="user", content=user_message)]

    if is_json:
        chat_response = client.chat(
            model=model, messages=messages, response_format={"type": "json_object"}
        )
    else:
        chat_response = client.chat(model=model, messages=messages)

    return chat_response.choices[0].message.content

In [ ]:
response = mistral(prompt)
print(response)

RAG + Function calling

In [6]:
def qa_with_context(text, question, chunk_size=512):
    # split document into chunks
    chunks = [text[i : i + chunk_size] for i in range(0, len(text), chunk_size)]
    # load into a vector database
    text_embeddings = np.array([get_text_embedding(chunk) for chunk in chunks])
    d = text_embeddings.shape[1]
    index = faiss.IndexFlatL2(d)
    index.add(text_embeddings)
    # create embeddings for a question
    question_embeddings = np.array([get_text_embedding(question)])
    # retrieve similar chunks from the vector database
    D, I = index.search(question_embeddings, k=2)
    retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
    # generate response based on the retrieve relevant text chunks

    prompt = f"""
    Context information is below.
    ---------------------
    {retrieved_chunk}
    ---------------------
    Given the context information and not prior knowledge, answer the query.
    Query: {question}
    Answer:
    """
    response = mistral(prompt)
    return response

In [7]:
I.tolist()

NameError: name 'I' is not defined

In [ ]:
I.tolist()[0]

In [ ]:
import functools

names_to_functions = {"qa_with_context": functools.partial(qa_with_context, text=text)}

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "qa_with_context",
            "description": "Answer user question by retrieving relevant context",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "user question",
                    }
                },
                "required": ["question"],
            },
        },
    },
]

In [ ]:
question = """
What are the ways AI can mitigate climate change in transportation?
"""

client = MistralClient(api_key=api_key, endpoint=dlai_endpoint)

response = client.chat(
    model="mistral-large-latest",
    messages=[ChatMessage(role="user", content=question)],
    tools=tools,
    tool_choice="any",
)

response

In [ ]:
tool_function = response.choices[0].message.tool_calls[0].function
tool_function

In [ ]:
tool_function.name

In [ ]:
import json

args = json.loads(tool_function.arguments)
args

In [ ]:
function_result = names_to_functions[tool_function.name](**args)
function_result